In [82]:
#Dependencies
import pandas as pd
import requests
import json
#from config import api_key
import matplotlib.pyplot as plt
import scipy.stats as sts
import datetime

In [83]:
#Airport test file
air_data = pd.read_csv('Resources/AirPortDataTemp.csv')
air_data.head()

,Unnamed: 0,Flight Date,IATA,State,Total Landed Flights
0,0,10/21/2020,JFK,New York,293
1,1,10/21/2020,SAN,California,37
2,2,10/21/2020,DFW,Texas,624
3,3,10/19/2020,JFK,New York,590
4,4,10/19/2020,SAN,California,183


In [84]:
#group airport data by date and state summing Total Landed Flights
group_air = air_data.groupby(['State', 'Flight Date'], as_index=False)['Total Landed Flights'].sum()
group_air.head()

,State,Flight Date,Total Landed Flights
0,California,10/14/2020,213
1,California,10/15/2020,192
2,California,10/16/2020,206
3,California,10/17/2020,210
4,California,10/18/2020,197


In [85]:
states_abbrev_file_path = "Resources/states_abbrev.csv"

states_abbrev_df = pd.read_csv(states_abbrev_file_path)
states_abbrev_df.head()

,State,Abbrev
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [86]:
#State Abbreviation in airport dataframe
abb_air_data = states_abbrev_df.merge(group_air, on='State')
abb_air_data.head()

,State,Abbrev,Flight Date,Total Landed Flights
0,California,CA,10/14/2020,213
1,California,CA,10/15/2020,192
2,California,CA,10/16/2020,206
3,California,CA,10/17/2020,210
4,California,CA,10/18/2020,197


In [104]:
# API Call for covid data 
covid_base_url = "https://api.covidtracking.com/v1/states/daily.json"
response = requests.get(covid_base_url).json()

#print(json.dumps(response, indent=4))

# pull the data from a source df to a target df by creating a new df

covid_df = pd.DataFrame(response, columns=['date', 'state', 'positive', 'death', 'hospitalized', 'totalTestResults'])
covid_df.head()

,date,state,positive,death,hospitalized,totalTestResults
0,20201021,AK,12644.0,68.0,NaN,547140.0
1,20201021,AL,174528.0,2805.0,19330.0,1273523.0
2,20201021,AR,101596.0,1751.0,6526.0,1242752.0
3,20201021,AS,0.0,0.0,NaN,1616.0
4,20201021,AZ,233912.0,5854.0,20820.0,1657389.0


In [108]:
covid_df['date'] = pd.to_datetime(covid_df['date'], format='%y%m%d')

ValueError: unconverted data remains: 021

In [106]:
covid_df.head()

,date,state,positive,death,hospitalized,totalTestResults
0,20201021,AK,12644.0,68.0,NaN,547140.0
1,20201021,AL,174528.0,2805.0,19330.0,1273523.0
2,20201021,AR,101596.0,1751.0,6526.0,1242752.0
3,20201021,AS,0.0,0.0,NaN,1616.0
4,20201021,AZ,233912.0,5854.0,20820.0,1657389.0


In [107]:
covid_df.dtypes

date                  int64
state                object
positive            float64
death               float64
hospitalized        float64
totalTestResults    float64
dtype: object

In [89]:
abb_air_data['Flight Date']=pd.to_datetime(abb_air_data['Flight Date'].astype(str), format='%m/%d/%Y')
abb_air_data.head()

,State,Abbrev,Flight Date,Total Landed Flights
0,California,CA,2020-10-14,213
1,California,CA,2020-10-15,192
2,California,CA,2020-10-16,206
3,California,CA,2020-10-17,210
4,California,CA,2020-10-18,197


In [90]:
abb_air_data.dtypes

State                           object
Abbrev                          object
Flight Date             datetime64[ns]
Total Landed Flights             int64
dtype: object

In [91]:
covid_df.dtypes

date                  int64
state                object
positive            float64
death               float64
hospitalized        float64
totalTestResults    float64
Formatted Date       object
dtype: object

In [92]:
covid_df['Formatted Date'] = pd.to_datetime(covid_df['Formatted Date'])
covid_df.dtypes

date                         int64
state                       object
positive                   float64
death                      float64
hospitalized               float64
totalTestResults           float64
Formatted Date      datetime64[ns]
dtype: object

In [93]:
final_df = pd.merge(covid_df, abb_air_data,  how='left', left_on=['state','Formatted Date'], right_on = ['Abbrev','Flight Date'])
final_df.head()

,date,state,positive,death,hospitalized,totalTestResults,Formatted Date,State,Abbrev,Flight Date,Total Landed Flights
0,20201021,AK,12644.0,68.0,NaN,547140.0,2020-01-22,NaN,NaN,NaT,NaN
1,20201021,AL,174528.0,2805.0,19330.0,1273523.0,2020-01-22,NaN,NaN,NaT,NaN
2,20201021,AR,101596.0,1751.0,6526.0,1242752.0,2020-01-22,NaN,NaN,NaT,NaN
3,20201021,AS,0.0,0.0,NaN,1616.0,2020-01-22,NaN,NaN,NaT,NaN
4,20201021,AZ,233912.0,5854.0,20820.0,1657389.0,2020-01-22,NaN,NaN,NaT,NaN


In [81]:
slt_df = final_df[final_df['state'] == 'CA'] 
slt_df.head()

,date,state,positive,death,hospitalized,totalTestResults,Formatted Date,State,Abbrev,Flight Date,Total Landed Flights
5,20201021,CA,877784.0,17027.0,NaN,17293139.0,2020-01-22,NaN,NaN,NaT,NaN
61,20201020,CA,874077.0,16992.0,NaN,17189070.0,2020-01-22,NaN,NaN,NaT,NaN
117,20201019,CA,870791.0,16970.0,NaN,17042408.0,2020-01-22,NaN,NaN,NaT,NaN
173,20201018,CA,867317.0,16943.0,NaN,16892062.0,2020-01-22,NaN,NaN,NaT,NaN
229,20201017,CA,864455.0,16899.0,NaN,16756711.0,2020-01-22,NaN,NaN,NaT,NaN


In [109]:
final_df.to_csv(r'Resources/Test_Data.csv')